<a href="https://colab.research.google.com/github/apal9569/Banking_Behaviour/blob/master/Banking_behaviour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd drive/My\ Drive

/content/drive/My Drive


In [0]:
!unzip d407ebe0db9a11e9.zip -d HDFC

Archive:  d407ebe0db9a11e9.zip
   creating: HDFC/DataSet/
  inflating: HDFC/DataSet/Train.csv  
  inflating: HDFC/DataSet/Sample_submission.csv  
  inflating: HDFC/DataSet/Test.csv   


In [0]:
cd HDFC/DataSet

/content/drive/My Drive/HDFC/DataSet


In [0]:
!ls

Sample_submission.csv  Test.csv  Train.csv


In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier

In [0]:
train=pd.read_csv('Train.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (746,835) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
train.shape

(17521, 2395)

In [0]:
train.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,...,Col2358,Col2359,Col2360,Col2361,Col2362,Col2363,Col2364,Col2365,Col2366,Col2367,Col2368,Col2369,Col2370,Col2371,Col2372,Col2373,Col2374,Col2375,Col2376,Col2377,Col2378,Col2379,Col2380,Col2381,Col2382,Col2383,Col2384,Col2385,Col2386,Col2387,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,RIGHGSN,1,1.086711e+04,8.648345e+03,1.576618e+04,2.890466e+04,5.273655e+03,2.346153e+04,1.285597e+04,2.871509e+04,9.114586e+03,3.043078e+04,9.293496,8.570479,10.063117,10.271758,0.548538,0.403366,NaN,NaN,1.486524,1.728324,0.993441,1.297050,0.137398,0.115987,0.260428,1.414214,1,1,1,1,1,1,0,6,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,214.0,214.0,68.0,22.0,22.0,7.0
1,RIMHI5ZGP,0,2.584312e+05,2.179633e+05,1.866287e+05,1.866287e+05,2.154519e+05,1.923615e+05,1.726538e+05,1.666974e+05,2.013518e+05,1.859404e+05,12.462385,12.280493,12.167132,12.136876,1.167898,2.676398,NaN,NaN,0.792123,0.934556,0.893203,0.966620,0.753892,0.808295,1.414214,NaN,1,1,3,4,4,0,4,5,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41.0,41.0,38.0,12.0,12.0,11.0
2,RIHOG1SZU,0,1.459355e+07,1.260060e+07,1.100315e+07,1.010315e+07,1.879882e+07,1.514035e+07,5.320778e+06,6.906324e+06,9.368338e+06,1.113330e+07,16.496090,16.749305,16.532874,16.128357,1.145182,2.203874,NaN,NaN,0.422264,0.498347,0.683582,0.735339,0.808115,0.776910,1.414214,NaN,3,3,4,12,4,0,2,4,NaN,1.0,NaN,NaN,...,1,0,1,0,1,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,1,0,378.0,378.0,310.0,21.0,21.0,16.0
3,RIHOLMQWU,0,6.727043e+05,8.181116e+05,6.794893e+05,6.794893e+05,8.257254e+05,7.605803e+05,3.577134e+05,4.236612e+05,4.304848e+05,4.065539e+05,13.419061,13.624018,13.541837,13.429097,1.204010,0.536304,NaN,NaN,0.437243,0.521341,0.623499,0.534531,-0.125829,0.388830,-1.414214,NaN,2,2,2,24,1,0,1,4,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1,2,0,1,2,0,0,0,0,0,1,2,0,0,0,0,0,1,2,0,0,0,0,0,0,702.0,702.0,644.0,951.0,951.0,743.0
4,RIHO584ET,0,-1.218742e+06,-3.005361e+06,-1.666241e+05,-2.403574e+06,2.583183e+06,2.086546e+06,5.938358e+06,5.446532e+06,3.473444e+06,3.262168e+06,NaN,14.764533,14.551021,NaN,NaN,NaN,NaN,NaN,NaN,1.344637,NaN,1.563430,-0.113101,0.617486,1.414214,NaN,4,3,4,68,8,0,2,4,NaN,3.0,NaN,NaN,...,0,0,0,0,0,0,2,1,0,0,0,0,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1305.0,1305.0,1211.0,2205.0,2205.0,1831.0


In [0]:
nan_col_before=train.columns[train.isna().any()].tolist()

In [0]:
num_nan_before=train[nan_col_before].isna().sum()

In [0]:
drop_col=train.columns[train.isna().sum()>1000].tolist()

In [0]:
train['Col2'].value_counts()

0    15760
1     1761
Name: Col2, dtype: int64

In [0]:
label=train['Col2']

In [0]:
train=train.drop(columns=['Col1','Col2'])

In [0]:
train=train.drop(columns=drop_col)

In [0]:
train.shape

(17521, 1635)

In [0]:
nan_col_after=train.columns[train.isna().any()].tolist()
num_nan_after=train[nan_col_after].isna().sum()


In [0]:
nan_col_after,num_nan_after

(['Col27'], Col27    682
 dtype: int64)

In [0]:
mean=train[nan_col_after].mean(axis=1)
train=train.fillna(method='ffill')
train.isna().any().sum()

0

In [0]:
model = ExtraTreesClassifier()
model.fit(train,label)
  
feat_importances = pd.Series(model.feature_importances_, index=train.columns)
imp_columns=feat_importances.nlargest(150).index

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
train1=train[imp_columns]

In [0]:
train1.head()

,Col36,Col27,Col127,Col160,Col51,Col579,Col193,Col393,Col186,Col196,Col927,Col383,Col49,Col161,Col83,Col130,Col54,Col1502,Col25,Col586,Col75,Col589,Col168,Col390,Col1412,Col169,Col72,Col12,Col205,Col917,Col2142,Col80,Col2232,Col8,Col3,Col82,Col10,Col59,Col582,Col135,...,Col1934,Col403,Col2008,Col1818,Col1322,Col395,Col1118,Col2009,Col1733,Col55,Col134,Col2002,Col121,Col7,Col90,Col1205,Col5,Col1712,Col66,Col402,Col1048,Col48,Col2190,Col133,Col1095,Col938,Col1781,Col1214,Col1851,Col69,Col65,Col1848,Col70,Col1665,Col1551,Col588,Col4,Col1186,Col1184,Col194
0,6,0.260428,1,14,2,0.333333,0.666667,4166.666667,0.333333,4166.666667,7,0.333333,2,14,1,3,1,0,0.137398,0.333333,1,5000.0,6.247142e+05,0.666667,0,6.247142e+05,3,3.043078e+04,12166.66667,22,0,2,0,2.346153e+04,1.086711e+04,2,2.871509e+04,2,0.000000,1,...,2,8.547600e+04,2,2,0,85475.99833,2,1,1,3,0,2,1,5.273655e+03,1,1,1.576618e+04,2,1,12166.66667,1,2,0,1,1,1,1,1,2,1,1,2,3,3,0,22.333333,8.648345e+03,2,2,32.666667
1,5,1.414214,0,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,11,0.000000,0,2,0,1,2,0,0.753892,0.000000,0,0.0,5.282476e+06,0.000000,0,5.282476e+06,2,1.859404e+05,0.00000,12,0,2,0,1.923615e+05,2.584312e+05,2,1.666974e+05,0,4.666667,0,...,0,1.117646e+05,0,0,0,0.00000,2,0,0,0,1,0,0,2.154519e+05,1,0,1.866287e+05,0,1,0.00000,1,2,0,0,0,0,0,0,1,0,0,2,2,1,1,5.666667,2.179633e+05,0,3,0.600000
2,4,1.414214,0,0,0,0.000000,1.200000,0.000000,0.000000,0.000000,16,0.000000,0,0,0,1,2,0,0.808115,1.666667,0,0.0,2.226136e+08,1.200000,0,2.226136e+08,1,1.113330e+07,31000.00000,21,0,1,0,1.514035e+07,1.459355e+07,1,6.906324e+06,0,63.666667,0,...,3,5.694258e+06,0,3,0,25704.00000,3,0,0,0,0,3,0,1.879882e+07,1,1,1.100315e+07,0,1,31000.00000,2,1,1,0,0,0,0,0,0,1,0,3,2,0,1,83.333333,1.260060e+07,0,3,12.800000
3,4,-1.414214,0,0,1,0.000000,0.000000,0.000000,0.000000,0.000000,743,0.000000,1,0,1,0,2,0,-0.125829,0.000000,1,0.0,2.821560e+07,0.000000,0,2.821560e+07,2,4.065539e+05,0.00000,951,0,2,0,7.605803e+05,6.727043e+05,2,4.236612e+05,1,49.000000,0,...,0,4.742903e+05,0,0,0,25582.30400,4,0,0,1,0,0,1,8.257254e+05,1,0,6.794893e+05,0,1,0.00000,1,1,1,0,0,0,0,0,0,1,0,4,2,0,2,77.333333,8.181116e+05,0,3,17.800000
4,4,1.414214,0,8,2,0.000000,3.600000,0.000000,0.000000,0.000000,1831,0.000000,2,8,1,1,1,2,-0.113101,6.000000,1,0.0,4.726991e+08,3.600000,2,4.726991e+08,1,3.262168e+06,121000.00000,2205,2,1,2,2.086546e+06,-1.218742e+06,1,5.446532e+06,2,56.666667,0,...,2,0.000000e+00,0,3,2,0.00000,2,0,0,2,0,1,0,2.583183e+06,1,0,-1.666241e+05,0,0,121000.00000,1,1,2,0,0,0,0,0,1,2,2,2,1,0,1,56.666667,-3.005361e+06,0,3,0.000000


In [0]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [0]:
trainx,valx,trainy,valy=train_test_split(train1,label,test_size=0.1)

In [0]:
model=XGBClassifier(max_depth=6,learning_rate=0.15,gamma=0.15)
model.fit(trainx,trainy)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.15,
              learning_rate=0.15, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(valx)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(valy, predictions)

In [0]:
accuracy

0.8881916714204221

In [0]:
test=pd.read_csv('Test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (700,731,740,752,761,789,811,820,829,841,850) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
test.shape

(20442, 2394)

In [0]:
test.head()

,Col1,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,Col22,Col23,Col24,Col25,Col26,Col27,Col28,Col29,Col30,Col31,Col32,Col33,Col34,Col35,Col36,Col37,Col38,Col39,Col40,Col41,...,Col2358,Col2359,Col2360,Col2361,Col2362,Col2363,Col2364,Col2365,Col2366,Col2367,Col2368,Col2369,Col2370,Col2371,Col2372,Col2373,Col2374,Col2375,Col2376,Col2377,Col2378,Col2379,Col2380,Col2381,Col2382,Col2383,Col2384,Col2385,Col2386,Col2387,Col2388,Col2389,Col2390,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397
0,RIGD58ZWD,8.167140e+04,8.614542e+04,7.532296e+04,7.532296e+04,1.125174e+05,1.002778e+05,2.038915e+04,3.053853e+04,8.395911e+04,8.138800e+04,11.310459,11.630863,11.515700,11.229540,1.143681,0.879545,NaN,NaN,0.236683,0.746188,0.405435,0.811625,0.086109,0.238817,0.150242,NaN,3,3,6,6,6,0,1,5,19.0,NaN,NaN,39.0,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44.0,44.0,23.0,41.0,41.0,36.0
1,RIH660YDS,-1.362824e+07,-4.493043e+06,-3.777257e+06,-4.493043e+06,-4.749764e+06,-4.749764e+06,1.147556e+07,1.147556e+07,1.272055e+07,1.272055e+07,NaN,NaN,NaN,NaN,NaN,-2.935756,-232.140528,-94.995287,NaN,NaN,NaN,NaN,-0.483522,-0.308259,-0.742190,NaN,6,5,7,19,9,0,2,6,2.0,0.0,NaN,5.0,4.0,...,0,0,1,0,0,0,2,2,1,1,0,0,1,2,1,1,2,1,1,1,2,1,3,1,2,1,0,0,1,1,1,1,2,2,1234.0,1234.0,680.0,154.0,153.0,86.0
2,RIH660Q96,1.302700e+06,1.341449e+06,1.240085e+06,1.341449e+06,1.477503e+06,1.477503e+06,3.904086e+05,3.904086e+05,4.979106e+05,4.979106e+05,14.079949,14.205864,14.205864,14.109261,1.081739,0.943850,2.122872,0.857057,0.291035,0.336995,0.291035,0.336995,-0.108229,-0.095389,-0.053413,NaN,2,2,2,3,1,0,1,6,NaN,NaN,NaN,NaN,5.0,...,0,0,0,0,0,0,0,0,1,0,2,1,0,1,0,0,0,0,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,33.0,24.0,19.0,3.0,1.0,1.0
3,RIYDO15W1,1.932258e+05,1.454433e+05,1.929148e+05,1.454433e+05,1.318250e+04,1.318250e+04,8.650108e+04,8.650108e+04,5.896693e+03,5.896693e+03,12.171615,9.486645,9.486645,11.887541,0.753925,1.978542,NaN,NaN,0.594741,0.447312,0.594741,0.447312,0.441902,0.383241,0.681768,NaN,4,4,5,5,5,0,1,6,6.0,5.0,NaN,1.0,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0,8.0,3.0,7.0,7.0,3.0
4,RIYBGC1ZD,3.913385e+05,1.987337e+05,1.703432e+05,1.987337e+05,3.046536e+05,3.046536e+05,2.528307e+05,2.528307e+05,3.034449e+05,3.034449e+05,12.877328,12.626931,12.626931,12.199721,1.166667,63.849949,NaN,NaN,1.272208,0.996032,1.272208,0.996032,0.507021,0.539030,0.742665,NaN,1,1,1,1,1,0,1,6,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.0,7.0,7.0,6.0,6.0,5.0


In [0]:
idx=test['Col1']
test=test.drop(columns=drop_col)

In [0]:
nan_col_after_test=test.columns[test.isna().any()].tolist()
num_nan_after_test=test[nan_col_after].isna().sum()

In [0]:
nan_col_after_test

['Col27']

In [0]:
mean=test[nan_col_after].mean(axis=1)
test=test.fillna(method='ffill')
test.isna().any().sum()

0

In [0]:
test=test[imp_columns]

In [0]:
y=model.predict(train1)

In [0]:
y=pd.DataFrame(y)

In [0]:
y.shape

(17521, 1)

In [0]:
submission=pd.DataFrame()

In [0]:
submission['Col1']=idx
submission['Col2']=y

In [0]:
submission.to_csv('submission.csv',index=False)


In [0]:
submission.shape

(20442, 2)

In [0]:
submission['Col2'].nunique()

2